In [1]:
%pip install -r requirements.txt --upgrade

  Using cached gspread-6.2.0-py3-none-any.whl.metadata (11 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import fooddatacentral as fdc
import pandas as pd
from eep153_tools.sheets import read_sheets

In [3]:
import numpy as np

In [4]:
apikey = "LUJvyyWCR8JkuTS6nSLkEdtiC70cbEfD7TCQbvab"  # Replace with a real key!  "DEMO_KEY" will be slow...

In [5]:
def nutrition(age, sex, activity_level = False):
    # possible additional args pregnant, activity_level
    """
    takes the following
    age: integer from 2 through 120
    sex: either "F" for female or "M" for male

    outputs:
    A series object indexed by nutrition category with the series being the values respective 
    to the age range from input
    """
    if age<2 or age > 120:
        return "not valid age"
    RDIs = read_sheets('https://docs.google.com/spreadsheets/d/1xqixhrAoDq9rWJf_FC3Y2eXdd010DTLPCS7JJMCfwP8/edit?usp=sharing', sheet="rda")
    RDIs = RDIs.set_index('Nutrient')
    if sex == 'F':
        fullsex = 'Female'
    else: fullsex = 'Male'
    if activity_level:
        active_energy = activity(age, sex, activity_level)
    if age<4:
        if activity_level:
            output = RDIs['Child_1_3']
            output[0] = active_energy
            return output
        return RDIs['C 1-3']
    else:
        agerange = find_age_range(age)
        if activity_level:
            output = RDIs[f'{fullsex}_{agerange}']
            output[0] = active_energy
            return output
        return RDIs[f'{fullsex}_{agerange}']

In [6]:
nutrition(25, 'F', 'Active')

NameError: name 'activity' is not defined

In [7]:
def activity(age, sex, activity_level):
    activities = read_sheets('https://docs.google.com/spreadsheets/d/1xqixhrAoDq9rWJf_FC3Y2eXdd010DTLPCS7JJMCfwP8/edit?usp=sharing', sheet="activity_levels")
    activities = activities.set_index('Age')
    if age>18:
        activedic = {
        '19-20': np.arange(19,21),
        '21-25': np.arange(21,26),
        '26-30': np.arange(26,31),
        '31-35': np.arange(31,36),
        '36-40': np.arange(36,41),
        '41-45': np.arange(41,46),
        '46-50': np.arange(46,51),
        '51-55': np.arange(51,56),
        '56-60': np.arange(56,61),
        '61-65': np.arange(61,66),
        '66-70': np.arange(66,71),
        '71-75': np.arange(71,76),
        '76 and up': np.arange(76,120)
        }
        ager = "None"
        for age_range, ages in activedic.items():
            if age in ages:
                ager = age_range
        return activities.loc[ager, f"{sex} - {activity_level}"]
    else:
        return activities.loc[ager, f"{sex} - {activity_level}"]
    

In [8]:
def find_age_range(age):
    agedict = {'1_3': np.arange(1,4),
          '4_8': np.arange(4,9),
          '9_13': np.arange(9,14),
          '14_18': np.arange(14,19),
          '19_30': np.arange(19,31),
          '31_50': np.arange(31,51),
          '51U': np.arange(51,120)}
    for age_range, ages in agedict.items():
        if age in ages:
            return age_range
    return "Age not in range"

In [24]:
RDIs

,Nutrition,Source,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
0,Energy,---,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
1,Protein,RDA,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
2,"Fiber, total dietary",---,14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
3,"Folate, DFE",RDA,150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
4,"Calcium, Ca",RDA,700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0
5,"Carbohydrate, by difference",RDA,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0
6,"Iron, Fe",RDA,7.0,10.0,10.0,8.0,8.0,15.0,11.0,18.0,8.0,18.0,8.0,8.0,8.0
7,"Magnesium, Mg",RDA,80.0,130.0,130.0,240.0,240.0,360.0,410.0,310.0,400.0,320.0,420.0,320.0,420.0
8,Niacin,RDA,6.0,8.0,8.0,12.0,12.0,14.0,16.0,14.0,16.0,14.0,16.0,14.0,16.0
9,"Phosphorus, P",RDA,460.0,500.0,500.0,1250.0,1250.0,1250.0,1250.0,700.0,700.0,700.0,700.0,700.0,700.0


In [22]:
RDIs = read_sheets('https://docs.google.com/spreadsheets/d/1c6jfP_Y5znM7hYYjuOgE1N6ttjLG8Qjbu03LdSDmHT8/edit?usp=sharing', sheet="diet_minimums")
pd.Series(RDIs.columns).str.extract('(\d+)')

,0
0,NaN
1,NaN
2,1
3,4
4,4
5,9
6,9
7,14
8,14
9,19


In [20]:
for column in pd.Series(RDIs.columns):
    numarray = column.str.extract('(\d+)')
    range = np.arange(numarray[0], numarray[1] + 1)
    print(range)

AttributeError: 'str' object has no attribute 'str'

In [ ]:
agemap

In [17]:
np.arange(1,4)

array([1, 2, 3])